In [3]:
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt

In [84]:
frequency_difference = 625
freq1 = 400
freq2 = freq1 + frequency_difference
freq3 = freq2 + frequency_difference
freq4 = freq3 + frequency_difference

In [5]:
def openWavFile(filename):
    sampling_rate, data = wavfile.read(filename)
    print(f"sampling rate: {sampling_rate}")

    length = data.shape[0] / sampling_rate
    print(f"length = {length}s")
    
    return sampling_rate, data

In [6]:
def extract_peak_frequency(data, sampling_rate):
    fft_data = np.fft.fft(data)
    freqs = np.fft.fftfreq(len(data))
    
    peak_coefficient = np.argmax(np.abs(fft_data))
    peak_freq = freqs[peak_coefficient]
    
    return abs(peak_freq * sampling_rate)

In [7]:
def getChunks(milliseconds, sampling_rate):
    #divide this extracted data into 50ms chunks, and find the peak or dominant frequency of each chunk
    #now if the chunk's peak frequency is 1200Hz, then it is 0 bit and if it's 1400Hz, then it is 1 bit.
    #group these bits together and that's the received data
    try:
        milliseconds = milliseconds/1000

        delay = int(milliseconds*sampling_rate)
        print(f"delay between tones: {delay} samples")
        chunks = []

        data_length = len(data) - len(data)%delay
        prev = 0

        for i in range(delay, data_length, delay):
            chunks.append(data[prev:i])
            prev = i
    except:
        pass
    
    
    return chunks

In [8]:
def extractData(chunks, sampling_rate):
    try:
        peak_freqs = []
        for chunk in chunks:
            peak_freqs.append(round(extract_peak_frequency(chunk, sampling_rate), 0))
    except:
        pass
    
    return peak_freqs

In [92]:
def extractBits(peak_freqs):
    bits = []
    foundStartSequence = False
    foundEndSequence = False
    bin_bits = ""
    for frequency in peak_freqs:
        if(frequency >= 1190 and frequency <= 1210 and foundStartSequence == False):
            foundStartSequence = True
            print("Start sequence found!")
        
        elif(frequency >= 2390 and frequency <= 2410 and foundEndSequence == False):
            foundEndSequence = True
            print("End sequence found!")
            break
        
        elif(frequency >= freq1-100.0 and frequency <= freq1+100.0):
            bin_bits += "00"
        
        elif(frequency >= freq2-100.0 and frequency <= freq2+100.0):
            bin_bits += "01"

        elif(frequency >= freq3-100.0 and frequency <= freq3+100.0):
            bin_bits += "10"

        elif(frequency >= freq4-100.0 and frequency <= freq4+100.0):
            bin_bits += "11"
            
    
    #for i in range(len(bits)):
    #    if (i+1)%4 == 0:
    #        s += str(bits[i])
    #        bin_bits.append(s)
    #        s = ""
    #    else:
    #        s += str(bits[i])

    print(bin_bits)
    return bin_bits

In [10]:
def decodeAscii(bin_string):
    """binary_int = int(bin_string, 2);
    byte_number = binary_int.bit_length() + 7 // 8
    binary_array = binary_int.to_bytes(byte_number, "big")
    ascii_text = "Bin string cannot be decoded"
    for enc in ['utf-8', 'ascii', 'ansi']:
        try:
            ascii_text = binary_array.decode(encoding=enc)
            break
        except:
            pass
    print(ascii_text)"""
    
    
    bin_to_str = "".join([chr(int(bin_string[i:i+8],2)) for i in range(0,len(bin_string),8)])

    return bin_to_str

In [93]:
filename = input("Enter the wave file with extension: ")
millis = int(input("Enter the delay between tones in milliseconds: "))
#now open the wav file
sampling_rate, data = openWavFile(filename + ".wav")

#extract the chunks, each chunk is of 10ms in size
chunks = getChunks(millis, sampling_rate)

sampling rate: 48000
length = 1.36s
delay between tones: 192 samples


In [94]:
#get the peak frequencies from the chunks
peak_freqs = extractData(chunks, sampling_rate)

In [95]:
bits = extractBits(peak_freqs)

011000100100011100110001011010000110001001111001010000100111001101100010010101110100011001110110010010010100011101111000011101000101100101010111001110000110011101010110001100100011100101110001011001000100011101010110011100100100100101001101010101110110001001100010010101110110110001101100011000110110110101010010001101100110000101010011010000100111001101100010010101110100011001110110010010010100011101111000011101000101100101010111001110000110011101100010010001110011000101101000011000100111100101000010010110000110001000110010011100000011000001011010010101110111001101100111011110000101101001110100011101000110000101010111010101100111100101011010010010000111000001110000


In [96]:
bin_string = ""

for i in bits:
    bin_string += i
    
#print(bin_string)

#split the binary string at spaces, because, I don't know how to decode a space character(0100000)
bin_splits = bin_string.split("0100000")

In [97]:
for s in bin_splits:
    print(decodeAscii(s), end=" ")
print("\nbG1hbyBsbWFvIGxtYW8gV29qdGVrIMWbbWllcmR6aSBsbWFvIGxtYW8gbG1hbyBXb2p0ZWsgxZttaWVyZHpp")

bG1hbyBsbWFvIGxtYW8gV29qdGVrIMWbbWllcmR6aSBsbWFvIGxtYW8gbG1hbyBXb2p0ZWsgxZttaWVyZHpp 
bG1hbyBsbWFvIGxtYW8gV29qdGVrIMWbbWllcmR6aSBsbWFvIGxtYW8gbG1hbyBXb2p0ZWsgxZttaWVyZHpp
